In [111]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures


In [112]:
# Paths to the training and test data (adjust if needed)
train_path = "Data/gsb-544-fall-2025-regression/train_new.csv"
test_path  = "Data/gsb-544-fall-2025-regression/test_new.csv"

train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)

display(train.head())
display(test.head())


,SalePrice,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,159000,531363010,80.0,9605,Pave,SawyerW,1Fam,1Story,7,6,...,1,1,3,6,1218,Typ,0,0,2009,WD
1,271900,906203120,90.0,14684,Pave,SawyerW,1Fam,1Story,7,7,...,2,0,3,7,2196,Typ,0,0,2009,WD
2,137500,916176030,NaN,14375,Pave,Timber,1Fam,SLvl,6,6,...,1,0,3,7,1344,Typ,233,0,2009,COD
3,248500,528180130,48.0,6472,Pave,NridgHt,TwnhsE,1Story,9,5,...,2,0,2,6,1456,Typ,0,0,2009,WD
4,167000,528290030,61.0,9734,Pave,Gilbert,1Fam,SLvl,7,5,...,2,1,3,7,1374,Typ,0,0,2009,WD


,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,907135180,60,8070,Pave,CollgCr,1Fam,1Story,4,5,1994,...,1,0,3,5,990,Typ,0,0,2007,WD
1,528181040,40,6792,Pave,NridgHt,TwnhsE,1Story,7,5,2005,...,2,0,2,6,1368,Typ,0,0,2006,New
2,528175010,44,6371,Pave,NridgHt,TwnhsE,1Story,7,5,2009,...,2,0,2,6,1358,Typ,0,0,2010,New
3,531379030,70,8304,Pave,SawyerW,1Fam,2Story,6,5,1997,...,2,1,3,7,1837,Typ,0,0,2006,WD
4,923275090,37,6951,Pave,Mitchel,1Fam,1Story,5,5,1984,...,1,0,3,5,923,Typ,0,0,2008,WD


In [113]:
display(train.describe())
display(test.describe())

,SalePrice,PID,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Screen Porch,Pool Area,Yr Sold
count,2197.000000,2.197000e+03,1835.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000,2197.000000
mean,182376.851161,7.127872e+08,69.549319,10268.070096,6.116523,5.552572,1972.030951,1.592626,0.369595,2.858898,6.470642,1507.873464,15.954028,2.789713,2007.793810
std,81168.157405,1.887751e+08,23.188224,7975.740885,1.429613,1.107019,30.450069,0.553347,0.500405,0.834769,1.591381,511.844406,56.495484,39.991010,1.314345
min,13100.000000,5.263011e+08,21.000000,1300.000000,1.000000,1.000000,1872.000000,0.000000,0.000000,0.000000,2.000000,334.000000,0.000000,0.000000,2006.000000
25%,130000.000000,5.284410e+08,58.500000,7500.000000,5.000000,5.000000,1954.000000,1.000000,0.000000,2.000000,5.000000,1142.000000,0.000000,0.000000,2007.000000
50%,163500.000000,5.354532e+08,69.000000,9488.000000,6.000000,5.000000,1975.000000,2.000000,0.000000,3.000000,6.000000,1452.000000,0.000000,0.000000,2008.000000
75%,215000.000000,9.071811e+08,80.000000,11660.000000,7.000000,6.000000,2002.000000,2.000000,1.000000,3.000000,7.000000,1762.000000,0.000000,0.000000,2009.000000
max,755000.000000,1.007100e+09,313.000000,215245.000000,10.000000,9.000000,2010.000000,4.000000,2.000000,8.000000,15.000000,5642.000000,576.000000,800.000000,2010.000000


,PID,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Screen Porch,Pool Area,Yr Sold
count,6.050000e+02,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.000000,605.00000,605.000000,605.000000,605.000000,605.000000,605.000000
mean,7.149952e+08,68.239669,9232.049587,6.039669,5.578512,1968.464463,1.484298,0.398347,2.82314,6.332231,1463.930579,17.181818,0.733884,2007.757025
std,1.885002e+08,23.886953,4280.948456,1.397164,1.147644,30.724704,0.535342,0.503298,0.80774,1.523666,484.398263,57.083360,18.051167,1.339222
min,5.263521e+08,21.000000,1484.000000,2.000000,1.000000,1880.000000,0.000000,0.000000,0.00000,3.000000,540.000000,0.000000,0.000000,2006.000000
25%,5.314500e+08,58.000000,7200.000000,5.000000,5.000000,1949.000000,1.000000,0.000000,2.00000,5.000000,1092.000000,0.000000,0.000000,2007.000000
50%,5.354530e+08,66.000000,9120.000000,6.000000,5.000000,1970.000000,1.000000,0.000000,3.00000,6.000000,1377.000000,0.000000,0.000000,2008.000000
75%,9.064771e+08,80.000000,10880.000000,7.000000,6.000000,1998.000000,2.000000,1.000000,3.00000,7.000000,1709.000000,0.000000,0.000000,2009.000000
max,9.241510e+08,313.000000,50271.000000,10.000000,9.000000,2010.000000,3.000000,2.000000,6.00000,12.000000,3820.000000,410.000000,444.000000,2010.000000


In [114]:
display(train.nunique())
display(test.nunique())

SalePrice         874
PID              2197
Lot Frontage      122
Lot Area         1549
Street              2
Neighborhood       28
Bldg Type           5
House Style         8
Overall Qual       10
Overall Cond        9
Year Built        116
Roof Style          6
Heating             6
Central Air         2
Electrical          5
Full Bath           5
Half Bath           3
Bedroom AbvGr       8
TotRms AbvGrd      13
Gr Liv Area      1118
Functional          8
Screen Porch       99
Pool Area          13
Yr Sold             5
Sale Type          10
dtype: int64

PID              605
Lot Frontage      99
Lot Area         489
Street             2
Neighborhood      26
Bldg Type          5
House Style        8
Overall Qual       9
Overall Cond       9
Year Built        95
Roof Style         6
Heating            3
Central Air        2
Electrical         4
Full Bath          4
Half Bath          3
Bedroom AbvGr      7
TotRms AbvGrd     10
Gr Liv Area      454
Functional         7
Screen Porch      47
Pool Area          2
Yr Sold            5
Sale Type          9
dtype: int64

In [115]:
display(train.isnull().sum())
display(test.isnull().sum())

SalePrice          0
PID                0
Lot Frontage     362
Lot Area           0
Street             0
Neighborhood       0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Roof Style         0
Heating            0
Central Air        0
Electrical         1
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
TotRms AbvGrd      0
Gr Liv Area        0
Functional         0
Screen Porch       0
Pool Area          0
Yr Sold            0
Sale Type          0
dtype: int64

PID              0
Lot Frontage     0
Lot Area         0
Street           0
Neighborhood     0
Bldg Type        0
House Style      0
Overall Qual     0
Overall Cond     0
Year Built       0
Roof Style       0
Heating          0
Central Air      0
Electrical       0
Full Bath        0
Half Bath        0
Bedroom AbvGr    0
TotRms AbvGrd    0
Gr Liv Area      0
Functional       0
Screen Porch     0
Pool Area        0
Yr Sold          0
Sale Type        0
dtype: int64

In [116]:
train['Lot Frontage'] = train['Lot Frontage'].fillna(train['Lot Frontage'].median())


In [117]:
train['Electrical'] = train['Electrical'].fillna(train['Electrical'].mode()[0])


In [118]:
y = np.log1p(train["SalePrice"])
X = train.drop(columns=["SalePrice", "PID"])
X_test = test.drop(columns=["PID"])



In [119]:
print("Train shape:", X.shape)
print("Test shape:", X_test.shape)

Train shape: (2197, 23)
Test shape: (605, 23)


In [120]:
# Feature engineering
def engineer_features(df):
    df = df.copy()
    
    # House age
    df["HouseAge"] = df["Yr Sold"] - df["Year Built"]
    
    # Total bathrooms (full + half*0.5)
    df["TotalBath"] = df["Full Bath"] + 0.5 * df["Half Bath"]
    
    # Total rooms including baths
    df["TotalRooms"] = df["TotRms AbvGrd"] + df["Full Bath"] + df["Half Bath"]
    
    # Quality x Area interaction
    df["Qual_x_Area"] = df["Overall Qual"] * df["Gr Liv Area"]
    
    # Area per room (avoid division by zero)
    df["AreaPerRoom"] = df["Gr Liv Area"] / (df["TotRms AbvGrd"] + 1)
    
    # Log-transform skewed numeric features (log1p handles zeros)
    for col in ["Lot Area", "Gr Liv Area", "Pool Area", "Screen Porch"]:
        if col in df.columns:
            df[col] = np.log1p(df[col])
    
    return df

X = engineer_features(X)
X_test = engineer_features(X_test)

print("Engineered train shape:", X.shape)
print("Engineered test shape:", X_test.shape)



Engineered train shape: (2197, 28)
Engineered test shape: (605, 28)


In [121]:
# Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numeric columns:", numeric_features)
print("Categorical columns:", categorical_features)

Numeric columns: ['Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'TotRms AbvGrd', 'Gr Liv Area', 'Screen Porch', 'Pool Area', 'Yr Sold', 'HouseAge', 'TotalBath', 'TotalRooms', 'Qual_x_Area', 'AreaPerRoom']
Categorical columns: ['Street', 'Neighborhood', 'Bldg Type', 'House Style', 'Roof Style', 'Heating', 'Central Air', 'Electrical', 'Functional', 'Sale Type']


In [122]:
numeric_transformer_plain = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

numeric_transformer_poly_full = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("scaler", StandardScaler())
    ]
)

numeric_transformer_poly_interact = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("poly", PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
        ("scaler", StandardScaler())
    ]
)


categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor_plain = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_plain, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

preprocessor_poly_full = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_poly_full, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

preprocessor_poly_interact = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_poly_interact, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [123]:
models = {
    "lr": (preprocessor_plain, LinearRegression()),
    "dt": (preprocessor_plain, DecisionTreeRegressor(random_state=0)),
    "knn": (preprocessor_plain, KNeighborsRegressor()),

    "poly_full": (preprocessor_poly_full, LinearRegression()),
    "poly_interact": (preprocessor_poly_interact, LinearRegression()),

    "ridge": (preprocessor_plain, Ridge()),
    "lasso": (preprocessor_plain, Lasso(max_iter=10000)),

    "ridge_poly": (preprocessor_poly_interact, Ridge())
}


In [124]:
params = {
    "dt": {
        "model__max_depth": [5, 10, 15, 20, None],
        "model__min_samples_leaf": [1, 2, 5, 10, 20],
        "model__min_samples_split": [2, 5, 10, 20]
    },
    "knn": {
        "model__n_neighbors": list(range(3, 25, 2)),
        "model__weights": ["uniform", "distance"],
        "model__metric": ["euclidean", "manhattan"]
    },
    "ridge": {
        "model__alpha": [0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0]
    },
    "lasso": {
        "model__alpha": [0.0005, 0.001, 0.01, 0.1, 1.0],
        "model__max_iter": [10000]
    },
    "ridge_poly": {
        "model__alpha": [0.001, 0.01, 0.1, 1.0, 3.0, 10.0, 30.0, 100.0]
    }
}


In [125]:
best_models = {}
cv_scores_mse = {}
cv_scores_rmse = {}

for name, (preproc, model) in models.items():

    pipe = Pipeline([
        ("pre", preproc),
        ("model", model)
    ])

    print(f"\n=== Tuning / evaluating {name} ===")

    if name in params:   # only DT and KNN use grids
        gs = GridSearchCV(
            estimator=pipe,
            param_grid=params[name],
            cv=5,
            scoring="neg_mean_squared_error",
            n_jobs=-1
        )
        gs.fit(X, y)
        best_models[name] = gs.best_estimator_
        best_mse = -gs.best_score_
    else:
        scores = cross_val_score(pipe, X, y, cv=5,
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)
        pipe.fit(X, y)
        best_models[name] = pipe
        best_mse = -scores.mean()

    cv_scores_mse[name] = best_mse
    cv_scores_rmse[name] = np.sqrt(best_mse)

    print(f"{name} CV MSE: {best_mse:.6f}")
    print(f"{name} CV RMSE: {cv_scores_rmse[name]:.6f}")


=== Tuning / evaluating lr ===
lr CV MSE: 0.019137
lr CV RMSE: 0.138338

=== Tuning / evaluating dt ===
dt CV MSE: 0.030325
dt CV RMSE: 0.174139

=== Tuning / evaluating knn ===
knn CV MSE: 0.024246
knn CV RMSE: 0.155712

=== Tuning / evaluating poly_full ===
poly_full CV MSE: 0.030166
poly_full CV RMSE: 0.173682

=== Tuning / evaluating poly_interact ===
poly_interact CV MSE: 0.023822
poly_interact CV RMSE: 0.154343

=== Tuning / evaluating ridge ===
ridge CV MSE: 0.018953
ridge CV RMSE: 0.137669

=== Tuning / evaluating lasso ===
lasso CV MSE: 0.018997
lasso CV RMSE: 0.137829

=== Tuning / evaluating ridge_poly ===
ridge_poly CV MSE: 0.018242
ridge_poly CV RMSE: 0.135065


In [126]:
best_name = min(cv_scores_mse, key=cv_scores_mse.get)  # smallest MSE
final_model = best_models[best_name]

print("\n=== Model comparison (CV RMSE on log target) ===")
for name in cv_scores_rmse:
    print(f"{name}: RMSE_log = {cv_scores_rmse[name]}")

print(f"\nBest model based on CV MSE: {best_name}")


=== Model comparison (CV RMSE on log target) ===
lr: RMSE_log = 0.13833823737654116
dt: RMSE_log = 0.17413931881303796
knn: RMSE_log = 0.15571202073849658
poly_full: RMSE_log = 0.17368221472737036
poly_interact: RMSE_log = 0.15434264842712808
ridge: RMSE_log = 0.13766876382675014
lasso: RMSE_log = 0.13782911013140076
ridge_poly: RMSE_log = 0.13506461840421136

Best model based on CV MSE: ridge_poly


In [127]:
final_model.fit(X, y)

,steps,"[('pre', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [128]:
y_test_log_pred = final_model.predict(X_test)
y_test_pred = np.expm1(y_test_log_pred)   # invert log1p


In [129]:
# Build submission
submission = pd.DataFrame({
    "PID": test["PID"],
    "SalePrice": y_test_pred
})

submission_path = "submission.csv"
submission.to_csv(submission_path, index=False)
print(f"Saved submission to {submission_path}")

Saved submission to submission.csv
